In [3]:
using Compat.Test
using QuCircuit
import Base:sparse
import QuCircuit: GateType, gate, PrimitiveBlock

In [4]:
################################################
#              Grover Search Block             #
################################################
# psi and oracle are needed
struct Reflect{N, T} <: PrimitiveBlock{N, T}
    state :: Vector{T}
end
Reflect(state::Vector{T}) where T = Reflect{log2i(length(state)), T}(state)
Reflect(psi::Register) = Reflect(statevec(psi))

# NOTE: this should not be matrix multiplication based
import QuCircuit: apply!
function apply!(r::Register, g::Reflect)
    @views r.state[:,:] .= 2* (g.state'*r.state) .* reshape(g.state, :, 1) - r.state
    r
end
# since julia does not allow call overide on AbstractGate.
(rf::Reflect)(reg::Register) = apply!(reg, rf)

import Base: show
function show(io::IO, g::Reflect{N, T}) where {N, T}
    print("Reflect(N = $N")
end

show (generic function with 290 methods)

In [5]:
reg0 = rand_state(3)
mirror = randn(1<<3)*im; mirror[:]/=norm(mirror)
rf = Reflect(mirror)
reg = copy(reg0)
apply!(reg, rf)

Default Register (CPU, Complex{Float64}):
    total: 3
    batch: 1
    active: 3

In [6]:
v0, v1 = vec(reg.state), vec(reg0.state)
@test rf.state'*v0 ≈ rf.state'*v1
@test v0-rf.state'*v0*rf.state ≈ -(v1-rf.state'*v1*rf.state)

Test Passed


In [7]:
# if basis[abs(locs)] == locs>0?1:0, then flip the sign.
inference_oracle(locs) = Z(abs(locs[end])) |> C(locs[1:end-1]...)

inference_oracle (generic function with 1 method)

In [8]:
oracle = inference_oracle([2,-1,3])(3)

control:
	total: 3
	[-1, 2] control
	QuCircuit.Z{Complex{Float64}} at 3

In [9]:
# ≈ method for Identity/PermuteMultiply/Sparse
# add and mimus between sparse matrices.
# alway use sorted CSC format.
v = ones(1<<3)
v[Int(0b110)+1] *= -1
@test sparse(oracle) ≈ Diagonal(v)

Test Passed


In [32]:
uniform_state(num_bit) = register(ones(Complex128, 1<<num_bit)/sqrt(1<<num_bit))

grover_step!(psi::Register) = rv1(oracle(psi))
num_grover_step(prob::Real) = Int(round(pi/4/sqrt(prob))) - 1

function GroverSearch(oracle, num_bit::Int; psi = uniform_state(num_bit))
    uni_reflect = Reflect(uniform_state(num_bit))
    # solve a real search problem
    num_iter = num_grover_step(1.0/(1<<num_bit))

    for i in 1:num_iter
        psi = uni_reflect(oracle(psi))
    end
    psi
end

GroverSearch (generic function with 1 method)

In [33]:
####### Construct Grover Search Using Reflection Block
num_bit = 12
oracle = inference_oracle(push!(collect(Int, 1:num_bit-1), num_bit))(num_bit)

psi = GroverSearch(oracle, 12)
target_state = zeros(1<<num_bit); target_state[end] = 1
@test isapprox(abs(statevec(psi)'*target_state), 1, atol=1e-3)

Test Passed


In [28]:
function indices_with(num_bit::Int, poss::Vector{Int}, vals::Vector{UInt}, basis::Vector{UInt})
    mask = bmask(poss)
    valmask = bmask(poss[vals.!=0])
    basis[(basis .& mask) .== valmask]
end
bmask(ibit::Vector{Int}) = reduce(+, zero(UInt), [one(UInt) << b for b in (ibit.-1)])
bmask(ibit::Int) = one(UInt) << (ibit-1)

bmask (generic function with 2 methods)

In [29]:
################################################
#                Doing Inference               #
################################################
target_space(oracle) = real(Diagonal(sparse(oracle)).diag) .< 0

function inference(psi::Register, evidense::Vector{Int}, num_iter::Int)
    oracle = inference_oracle(evidense)(nqubit(psi))
    ts = target_space(oracle)
    rv1 = Reflect(copy(psi))
    grover = chain(oracle, rv1)
    for i in 1:num_iter
        p_target = norm(statevec(psi)[ts])^2
        println("step $i, overlap = $p_target")
        grover(psi)
    end
    psi
end

# the second version

inference (generic function with 1 method)

In [34]:
# test inference
num_bit = 12
psi0 = rand_state(num_bit)
#psi0 = uniform_state(num_bit)
evidense = [1, 2, 3, 4, 5, 6, 7, 8, 9]
#evidense = collect(1:num_bit)

# the desired subspace
basis = collect(UInt, 0:1<<num_bit-1)
subinds = indices_with(num_bit, abs.(evidense), UInt.(evidense.>0), basis)

v_desired = statevec(psi0)[subinds+1]
p = norm(v_desired)^2
v_desired[:] ./= sqrt(p)

# search the subspace
num_iter = num_grover_step(p)
println("Estimated num_step = ", pi/4/sqrt(p))
psi = inference(psi0, evidense, num_iter)
println((psi.state[subinds+1]'*v_desired) |> abs2)

Estimated num_step = 14.097256465070478
step 1, overlap = 0.003103920299949686
step 2, overlap = 0.027704537456708447
step 3, overlap = 0.07568783025444918
step 4, overlap = 0.14467821421130111
step 5, overlap = 0.23126007368335585
step 6, overlap = 0.33114686413019734
step 7, overlap = 0.4393933328295995
step 8, overlap = 0.5506403512960796
step 9, overlap = 0.659380238098992
step 10, overlap = 0.7602294363253448
step 11, overlap = 0.8481950458145451
step 12, overlap = 0.918922014533759
step 13, overlap = 0.9689087510009813
0.9956804830918514
